In [48]:
library("mvnormtest")
library("car")
library("IDPmisc")
library("dplyr")
library("rcompanion")


Attaching package: 'dplyr'


The following object is masked from 'package:car':

    recode


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [4]:
heartattacks <- read.csv("C:/Users/Marcy/Documents/Entity Coursework/DSO 105 Basic Stats in Python/Lesson 8 MANOVAS/heartAttacks.csv")

## It is well-known that men are more likely to have heart attacks than women. How does gender (sex) influence some of the heart attack predictors like resting blood pressure (trestbps) and cholesterol (chol)?

#### data Wrangling first

In [30]:
str(heartattacks$trestbps)

 int [1:303] 145 130 130 120 120 140 140 120 172 150 ...


In [31]:
str(heartattacks$chol)

 int [1:303] 233 250 204 236 354 192 294 263 199 168 ...


#### both integers. good. 

In [32]:
tail(heartattacks)

,ï..age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>
298,59,1,0,164,176,1,0,90,0,1.0,1,2,1,0
299,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
300,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
301,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
302,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
303,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


#### sample size assumption is met

#### subset the two dependent variables

In [33]:
keep <- c("trestbps", "chol")
heartattacks1 <- heartattacks[keep]

#### no need to limit the number of rows here. 

#### format as a matrix

In [34]:
heartattacks2 <- as.matrix(heartattacks1)

#### test assumptions - already got sample size. next is multivariate normality

In [35]:
mshapiro.test(t(heartattacks2))


	Shapiro-Wilk normality test

data:  Z
W = 0.94568, p-value = 3.93e-09


#### p value is significant >.05, but we'll proceed with Homogeneity of Variance

In [37]:
heartattacks2$trestbps <- as.numeric(heartattacks2 ~ trestbps)

ERROR: Error in eval(expr, envir, enclos): 'language' object cannot be coerced to type 'double'


In [38]:
heartattacks2$trestbps <- as.numeric(heartattacks2 ~ trestbps)

ERROR: Error in eval(expr, envir, enclos): 'language' object cannot be coerced to type 'double'


In [39]:
heartattacks2$trestbps <- as.factor(heartattacks2 ~ trestbps)


ERROR: Error in unique.default(x, nmax = nmax): unique() applies only to vectors


In [40]:
heartattacks$chol <- as.factor(heartattacks ~ chol)

ERROR: Error in unique.default(x, nmax = nmax): unique() applies only to vectors


In [41]:
leveneTest(trestbps ~ sex, data=heartattacks)

ERROR: Error in leveneTest.formula(trestbps ~ sex, data = heartattacks): Levene's test is not appropriate with quantitative explanatory variables.


### FYI LeveneTest has thrown an error EVERY time I've run it in lessons 7 and 8. we don't know why.

In [42]:
leveneTest(chol ~ sex, data=heartattacks)

ERROR: Error in leveneTest.formula(chol ~ sex, data = heartattacks): Levene's test is not appropriate with quantitative explanatory variables.


#### absence of multicollinearity

In [43]:
cor.test(heartattacks$trestbps, heartattacks$chol, method = "pearson", use="complete.obs")


	Pearson's product-moment correlation

data:  heartattacks$trestbps and heartattacks$chol
t = 2.1534, df = 301, p-value = 0.03208
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.01064389 0.23262366
sample estimates:
      cor 
0.1231742 


### with a correlation of .12, that's <.7, so the dependent variables are not strongly correlated - in other words there isn't a significant relationship between the dependent variables, so there IS an absence of multicollinearity and the assumption is met. 

### Now to run the MANOVA

In [44]:
MANOVA <- manova(cbind(trestbps, chol) ~ sex, data=heartattacks)
summary(MANOVA)

           Df   Pillai approx F num Df den Df   Pr(>F)   
sex         1 0.040235   6.2882      2    300 0.002112 **
Residuals 301                                            
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

### bound the dependent variables together with cbind. in the summary, there isn't a significant difference in resting blood pressure and cholesterol between the sexes, but a moderate one. I'll run a post hoc to see which way that difference lies.

In [45]:
summary.aov(MANOVA, test = "wilks")

 Response trestbps :
             Df Sum Sq Mean Sq F value Pr(>F)
sex           1    299  299.36  0.9732 0.3247
Residuals   301  92592  307.61               

 Response chol :
             Df Sum Sq Mean Sq F value  Pr(>F)    
sex           1  31778   31778  12.271 0.00053 ***
Residuals   301 779523    2590                    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


### oh, cholesterol IS affected by gender, by quite a bit more than resting blood pressure is affected. Male or female? I'll run a mean to see. 

In [55]:
gendermeans <- heartattacks %>% group_by(chol) %>% summarize(Mean = mean(sex))

In [56]:
gendermeans

chol,Mean
<int>,<dbl>
126,1.00
131,1.00
141,0.00
149,0.50
157,1.00
160,0.00
164,0.00
166,1.00
167,1.00
